In [1]:
from theano import tensor as T
import lasagne_dev as nn
import numpy as np
import pickle
from input_process import read_images
import random
import math
import cmath
from g_gradient import *
import timeit
import unet

/usr/local/lib/python2.7/dist-packages/theano/gpuarray/dnn.py:135: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to version 5.1.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 6020 on context None
Mapped name None to device cuda: Quadro K2000 (0000:01:00.0)


In [2]:
start = timeit.default_timer()
epoch = 1

input_var = T.tensor4('input_var')  
label_var = T.tensor4('label_var')

[network, loss, test_loss, test_acc, output_det] = unet.network(input_var, label_var, [5,1,116,116])


params = nn.layers.get_all_params(network)
lr = theano.shared(nn.utils.floatX(1e-4)) # learning rate
updates = nn.updates.adam(loss,params, learning_rate=lr) # adam most widely used update scheme
#updates = nn.updates.momentum(loss,params, learning_rate=lr,momentum=0.99)
gs = nn.layers.get_all_gs(network)

stop = timeit.default_timer()
print stop - start 

13.7809448242


In [3]:
def g_psi(x,y,params):
    f,gamma,sigma,theta,psi = params
    value = gabor_filter(x,y,params)  
    return value

def gabor_filter_update_dev(size, params, f):
    bond = math.floor(size/2)
    x_range = np.linspace(-bond, bond, size)
    y_range = np.linspace(-bond, bond, size)

    [x_range,y_range] = list(map(lambda x:x.reshape(1,-1),np.meshgrid(x_range,y_range)))
    gfilter = []
    updates = []
    for (x,y) in zip(np.ndarray.tolist(x_range)[0], np.ndarray.tolist(y_range)[0]):        
        update = f(x,y,params)
        updates.append(update.real)
    updates = np.array(updates, dtype=np.float32).reshape(size,-1)
    return updates


def g_updates_dev(loss, params, gs):
    # Calculate gradients: 0.05
    gs_gradients = []
    gradients = nn.updates.get_or_compute_grads(loss, params)
    
    for w_index in range (0, int(len(params)/2)):
        # First Loop
        # print(w_index)
        
        ws = params[w_index*2]
        g_params = gs[w_index].get_value()
        ws_gradients = gradients[w_index*2]

        [num_filters, num_channels, filter_size, filter_size] = ws.get_value().shape

        g_gradients = np.array([], dtype=np.float32).reshape(1,-1)
    

        # Second and third Loop

        for i in range (0, num_filters):
            for j in range (0, num_channels):
                g = g_params[i,j,:]
                w = ws[i,j,:,:]

                w_gradient = ws_gradients[i,j,:]

                #addition = gabor_filter_update_dev(filter_size, g, g_psi)	# need adjustment

                
                #a = (w_gradient*addition).sum()
                #psi_gradient = a/9
                #g_gradient = np.array([0,0,0,0,psi_gradient.eval()]).reshape(1,-1)		# need adjustment				
                #g_gradients = np.concatenate((g_gradients,g_gradient),axis=1)
                
                #a = sum(sum(w_gradient*addition))
                #a = (w_gradient*addition).sum()
                #psi_gradient = a/9
                g_gradient = np.array([0,0,0,0,0]).reshape(1,-1)
                #g_gradient = np.array([0,0,0,0,psi_gradient]).reshape(1,-1)
                g_gradients = np.concatenate((g_gradients,g_gradient),axis=1)

        g_gradients = g_gradients.reshape([num_filters,num_channels,-1])
        
        #g_gradients = theano.shared(nn.utils.floatX(g_gradients))
        g_gradients = theano.shared(g_gradients)

        gs_gradients.append(g_gradients)
    

    gs_updates = nn.updates.adam_dev(gs_gradients, gs)
    return 0

In [4]:
start = timeit.default_timer()
gs_updates = g_updates_dev(loss, params, gs)
stop = timeit.default_timer()
print stop - start 

42.0458910465


In [5]:
start = timeit.default_timer()
w_index = 0
ws = params[w_index*2]
g_params = gs[w_index].get_value()
ws_gradients = gradients[w_index*2]

[num_filters, num_channels, filter_size, filter_size] = ws.get_value().shape

g_gradients = np.array([], dtype=np.float32).reshape(1,-1)

i = 0
j = 0
g = g_params[i,j,:]
w = ws[i,j,:,:]

w_gradient = ws_gradients[i,j,:]

addition = gabor_filter_update_dev(filter_size, g, g_psi)	# need adjustment
stop = timeit.default_timer()
print stop - start 

NameError: name 'gradients' is not defined